<a href="https://colab.research.google.com/github/thisissamuca/Webscraping/blob/main/Aula_WS_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **WebScraping para obtenção de Imagens - part 2**

## **Instalando bibliotecas**

In [ ]:
pip install requests

In [ ]:
pip install beautifulsoup4

In [ ]:
pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## **Importando bibliotecas necessárias**

In [1]:
import wikipedia
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import random
import time
import re

# **Resumo da biblioteca Wikipedia**

## **Obtendo o resumo de qualquer título**

In [ ]:
wikipedia.summary("Mineral", sentences = 1)  

'In geology and mineralogy, a mineral or mineral species is, broadly speaking, a solid chemical compound with a fairly well-defined chemical composition and a specific crystal structure that occurs naturally in pure form.The geological definition of mineral normally excludes compounds that occur only in living beings.'

## **Pesquisando título e sugestões**

In [ ]:
wikipedia.search("Mineral", results = 5) 

['Mineral',
 'Mineral oil',
 'Mineral (nutrient)',
 'Mineralization',
 'Mineral wool']

## **Obtendo dados completos da página da Wikipedia**

In [ ]:
page = wikipedia.page("Mineral") 

#print(page.html) 
#print(page.original_title) 
#print(page.links[0:10]) 

## **Alterando o idioma da página da Wikipedia**

In [ ]:
wikipedia.set_lang("pt") 

wikipedia.summary("Mineral", sentences = 1)  

# **Obtendo imagens da página Wikipedia**

In [2]:
# Chamando uma base de dados criada anteriormente

df = pd.read_csv('links_minerals.csv')

In [15]:
# Criando uma lista para armazenas as urls

url_img = []

url_img.clear()

Agora precisamos criar um laço para obter os links em função dos nomes do nosso dataframe. Entretanto, precisaremos utilizar várias excessões:

In [16]:
import time

# Criando um laço para acessar as imagens de todos elementos da base de dados

inicio = time.time()

for name in df['name']:

  try:
    page = wikipedia.page(name, auto_suggest=False)
    img = page.images
    url_img.append(img)

  except wikipedia.exceptions.PageError as e:
    pass

  except wikipedia.DisambiguationError as f:
     for name in f.options:
        try:
          page = wikipedia.page(name)
          img = page.images
          url_img.append(img)

        except wikipedia.exceptions.DisambiguationError as g:
          pass

        except wikipedia.exceptions.PageError as h:
          pass

        except KeyError as i:
          pass

  except:
    pass

fim = time.time()

print(fim - inicio)

#len(url_img)

/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


1201.880602836609


### **Separando os links inúteis**

Podemos fazer isso de duas formas. A primeira pode apresentar algumas limitações, no entanto pode ser mais vantajosa em alguns casos. Vamos separar uma lista de termos que não queremos, faremos isso seguindo uma lógica: os termos que separei eles se repetem ao longo dos objetos dentro da lista, todos eles tem o padrão de extensão .svg

In [17]:
terms = [
'https://upload.wikimedia.org/wikipedia/en/4/4a/Commons-logo.svg',
'https://upload.wikimedia.org/wikipedia/commons/9/99/Wiktionary-logo-en-v2.svg',
'https://upload.wikimedia.org/wikipedia/en/8/8a/OOjs_UI_icon_edit-ltr-progressive.svg',
'https://upload.wikimedia.org/wikipedia/commons/4/4c/Wikisource-logo.svg',
'https://upload.wikimedia.org/wikipedia/commons/a/a4/Text_document_with_red_question_mark.svg',
'https://upload.wikimedia.org/wikipedia/en/d/db/Symbol_list_class.svg',
'https://upload.wikimedia.org/wikipedia/commons/9/99/Wiktionary-logo-en-v2.svg',
'https://upload.wikimedia.org/wikipedia/en/0/06/Wiktionary-logo-v2.svg',
'https://upload.wikimedia.org/wikipedia/en/9/94/Symbol_support_vote.svg',
'https://upload.wikimedia.org/wikipedia/en/9/96/Symbol_category_class.svg',
'https://upload.wikimedia.org/wikipedia/en/1/1b/Semi-protection-shackle.svg',
'https://upload.wikimedia.org/wikipedia/en/8/8a/OOjs_UI_icon_edit-ltr-progressive.svg',
'https://upload.wikimedia.org/wikipedia/commons/0/0b/Wikiversity_logo_2017.svg',
'https://upload.wikimedia.org/wikipedia/commons/e/ed/Papapishu-Lab-icon-6.svg',
'https://upload.wikimedia.org/wikipedia/commons/8/8a/Loudspeaker.svg',
'https://upload.wikimedia.org/wikipedia/commons/b/b5/Radioactive.svg',
'https://upload.wikimedia.org/wikipedia/commons/4/47/Sound-icon.svg',
'https://upload.wikimedia.org/wikipedia/en/9/99/Question_book-new.svg',
'https://upload.wikimedia.org/wikipedia/en/f/f2/Edit-clear.svg',
'https://upload.wikimedia.org/wikipedia/en/f/fb/Yes_check.svg',
'https://upload.wikimedia.org/wikipedia/commons/d/dd/Wikivoyage-Logo-v3-icon.svg',
'https://upload.wikimedia.org/wikipedia/en/6/62/PD-icon.svg',
'https://upload.wikimedia.org/wikipedia/commons/f/fa/Wikibooks-logo.svg',
'https://upload.wikimedia.org/wikipedia/commons/f/fa/Wikiquote-logo.svg',
'https://upload.wikimedia.org/wikipedia/commons/2/24/Wikinews-logo.svg',
'https://upload.wikimedia.org/wikipedia/commons/1/1c/Wiki_letter_w_cropped.svg',
'https://upload.wikimedia.org/wikipedia/en/6/6c/Wiki_letter_w.svg',
'https://upload.wikimedia.org/wikipedia/en/b/b4/Ambox_important.svg',
'https://upload.wikimedia.org/wikipedia/commons/1/1b/Ambox_question.svg',
'https://upload.wikimedia.org/wikipedia/commons/9/98/Ambox_current_red.svg',
'https://upload.wikimedia.org/wikipedia/commons/9/96/Steady2.svg',
'https://upload.wikimedia.org/wikipedia/commons/b/b0/Openstreetmap_logo.svg',
'https://upload.wikimedia.org/wikipedia/commons/5/59/Increase_Negative.svg',
'https://upload.wikimedia.org/wikipedia/commons/b/b0/Increase2.svg',
'https://upload.wikimedia.org/wikipedia/commons/9/92/Decrease_Positive.svg',
'https://upload.wikimedia.org/wikipedia/commons/e/ed/Decrease2.svg',
'https://upload.wikimedia.org/wikipedia/commons/7/77/Flag_of_Algeria.svg',
'https://upload.wikimedia.org/wikipedia/commons/1/1a/Flag_of_Argentina.svg',
'https://upload.wikimedia.org/wikipedia/commons/8/88/Flag_of_Australia_%28converted%29.svg',
'https://upload.wikimedia.org/wikipedia/commons/4/41/Flag_of_Austria.svg',
'https://upload.wikimedia.org/wikipedia/commons/2/2c/Flag_of_Bahrain.svg',
'https://upload.wikimedia.org/wikipedia/commons/9/92/Flag_of_Belgium_%28civil%29.svg',
'https://upload.wikimedia.org/wikipedia/commons/9/9c/Flag_of_Brunei.svg',
'https://upload.wikimedia.org/wikipedia/commons/9/9a/Flag_of_Bulgaria.svg',
'https://upload.wikimedia.org/wikipedia/commons/d/d9/Flag_of_Canada_%28Pantone%29.svg',
'https://upload.wikimedia.org/wikipedia/commons/7/78/Flag_of_Chile.svg',
'https://upload.wikimedia.org/wikipedia/commons/2/21/Flag_of_Colombia.svg',
'https://upload.wikimedia.org/wikipedia/commons/1/1b/Flag_of_Croatia.svg',
'https://upload.wikimedia.org/wikipedia/commons/d/d4/Flag_of_Cyprus.svg',
'https://upload.wikimedia.org/wikipedia/commons/9/9c/Flag_of_Denmark.svg',
'https://upload.wikimedia.org/wikipedia/commons/3/34/Flag_of_Djibouti.svg',
'https://upload.wikimedia.org/wikipedia/commons/f/fe/Flag_of_Egypt.svg',
'https://upload.wikimedia.org/wikipedia/commons/8/8f/Flag_of_Estonia.svg',
'https://upload.wikimedia.org/wikipedia/commons/b/bc/Flag_of_Finland.svg',
'https://upload.wikimedia.org/wikipedia/commons/0/02/Flag_of_Gibraltar.svg',
'https://upload.wikimedia.org/wikipedia/commons/5/5c/Flag_of_Greece.svg',
'https://upload.wikimedia.org/wikipedia/commons/8/82/Flag_of_Honduras.svg',
'https://upload.wikimedia.org/wikipedia/commons/c/c1/Flag_of_Hungary.svg',
'https://upload.wikimedia.org/wikipedia/commons/c/ce/Flag_of_Iceland.svg',
'https://upload.wikimedia.org/wikipedia/commons/f/f6/Flag_of_Iraq.svg',
'https://upload.wikimedia.org/wikipedia/commons/4/45/Flag_of_Ireland.svg',
'https://upload.wikimedia.org/wikipedia/commons/d/d4/Flag_of_Israel.svg',
'https://upload.wikimedia.org/wikipedia/commons/c/c0/Flag_of_Jordan.svg',
'https://upload.wikimedia.org/wikipedia/commons/a/aa/Flag_of_Kuwait.svg',
'https://upload.wikimedia.org/wikipedia/commons/8/84/Flag_of_Latvia.svg',
'https://upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Arkansas.svg',
'https://upload.wikimedia.org/wikipedia/commons/d/d4/Flag_of_Cyprus.svg',
'https://upload.wikimedia.org/wikipedia/commons/5/5c/Flag_of_Greece.svg',
'https://upload.wikimedia.org/wikipedia/commons/d/d9/Flag_of_Norway.svg',
'https://upload.wikimedia.org/wikipedia/commons/a/ab/Flag_of_Panama.svg',
'https://upload.wikimedia.org/wikipedia/commons/8/88/Flag_of_Australia_%28converted%29.svg',
'https://upload.wikimedia.org/wikipedia/commons/d/d9/Flag_of_Canada_%28Pantone%29.svg',
'https://upload.wikimedia.org/wikipedia/commons/d/d3/Flag_of_Kazakhstan.svg',
'https://upload.wikimedia.org/wikipedia/commons/c/c7/Flag_of_Kyrgyzstan.svg',
'https://upload.wikimedia.org/wikipedia/commons/5/56/Flag_of_Laos.svg',
'https://upload.wikimedia.org/wikipedia/commons/f/fc/Flag_of_Mexico.svg',
'https://upload.wikimedia.org/wikipedia/commons/8/8c/Flag_of_Myanmar.svg',
'https://upload.wikimedia.org/wikipedia/commons/a/af/Flag_of_South_Africa.svg',
'https://upload.wikimedia.org/wikipedia/commons/d/d0/Flag_of_Tajikistan.svg',
'https://upload.wikimedia.org/wikipedia/commons/a/a9/Flag_of_Thailand.svg',
'https://upload.wikimedia.org/wikipedia/commons/b/b4/Flag_of_Turkey.svg',
'https://upload.wikimedia.org/wikipedia/commons/f/fa/Flag_of_the_People%27s_Republic_of_China.svg',
'https://upload.wikimedia.org/wikipedia/commons/b/b8/Flag_of_Cornwall.svg',
'https://upload.wikimedia.org/wikipedia/en/f/f3/Flag_of_Russia.svg',
'https://upload.wikimedia.org/wikipedia/en/9/9e/Flag_of_Japan.svg',
'https://upload.wikimedia.org/wikipedia/commons/4/47/Flag_of_Liechtenstein.svg',
'https://upload.wikimedia.org/wikipedia/commons/1/11/Flag_of_Lithuania.svg',
'https://upload.wikimedia.org/wikipedia/commons/d/da/Flag_of_Luxembourg.svg',
'https://upload.wikimedia.org/wikipedia/commons/7/73/Flag_of_Malta.svg',
'https://upload.wikimedia.org/wikipedia/commons/7/77/Flag_of_Mauritius.svg',
'https://upload.wikimedia.org/wikipedia/commons/d/d0/Flag_of_Mozambique.svg',
'https://upload.wikimedia.org/wikipedia/commons/3/3e/Flag_of_New_Zealand.svg',
'https://upload.wikimedia.org/wikipedia/commons/7/79/Flag_of_North_Macedonia.svg',
'https://upload.wikimedia.org/wikipedia/commons/d/d9/Flag_of_Norway.svg',
'https://upload.wikimedia.org/wikipedia/commons/d/dd/Flag_of_Oman.svg',
'https://upload.wikimedia.org/wikipedia/commons/5/5c/Flag_of_Portugal.svg',
'https://upload.wikimedia.org/wikipedia/commons/6/65/Flag_of_Qatar.svg',
'https://upload.wikimedia.org/wikipedia/commons/7/73/Flag_of_Romania.svg',
'https://upload.wikimedia.org/wikipedia/commons/0/0d/Flag_of_Saudi_Arabia.svg',
'https://upload.wikimedia.org/wikipedia/commons/f/ff/Flag_of_Serbia.svg',
'https://upload.wikimedia.org/wikipedia/commons/f/fc/Flag_of_Seychelles.svg',
'https://upload.wikimedia.org/wikipedia/commons/e/e6/Flag_of_Slovakia.svg',
'https://upload.wikimedia.org/wikipedia/commons/f/f0/Flag_of_Slovenia.svg',
'https://upload.wikimedia.org/wikipedia/commons/a/af/Flag_of_South_Africa.svg',
'https://upload.wikimedia.org/wikipedia/commons/0/09/Flag_of_South_Korea.svg',
'https://upload.wikimedia.org/wikipedia/commons/f/f3/Flag_of_Switzerland.svg',
'https://upload.wikimedia.org/wikipedia/commons/b/b4/Flag_of_Turkey.svg',
'https://upload.wikimedia.org/wikipedia/commons/f/fe/Flag_of_Uruguay.svg',
'https://upload.wikimedia.org/wikipedia/commons/c/cb/Flag_of_the_Czech_Republic.svg',
'https://upload.wikimedia.org/wikipedia/commons/2/20/Flag_of_the_Netherlands.svg',
'https://upload.wikimedia.org/wikipedia/commons/f/fc/Flag_of_Seychelles.svg',
'https://upload.wikimedia.org/wikipedia/commons/7/72/Flag_of_the_Republic_of_China.svg',
'https://upload.wikimedia.org/wikipedia/en/b/ba/Flag_of_Germany.svg',
'https://upload.wikimedia.org/wikipedia/commons/8/85/Pfeil_links.svg',
'https://upload.wikimedia.org/wikipedia/commons/7/76/Pfeil_oben.svg',
'https://upload.wikimedia.org/wikipedia/commons/c/c5/Pfeil_rechts.svg',
'https://upload.wikimedia.org/wikipedia/commons/c/c3/Pfeil_unten.svg',
'https://upload.wikimedia.org/wikipedia/en/0/05/Flag_of_Brazil.svg',
'https://upload.wikimedia.org/wikipedia/en/c/c3/Flag_of_France.svg',
'https://upload.wikimedia.org/wikipedia/commons/0/04/Flag_of_Gabon.svg',
'https://upload.wikimedia.org/wikipedia/en/0/03/Flag_of_Italy.svg',
'https://upload.wikimedia.org/wikipedia/en/1/12/Flag_of_Poland.svg',
'https://upload.wikimedia.org/wikipedia/en/9/9a/Flag_of_Spain.svg',
'https://upload.wikimedia.org/wikipedia/en/4/4c/Flag_of_Sweden.svg',
'https://upload.wikimedia.org/wikipedia/en/a/ae/Flag_of_the_United_Kingdom.svg',
'https://upload.wikimedia.org/wikipedia/commons/f/fe/Flag_of_Saint_Kitts_and_Nevis.svg',
'https://upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Arizona.svg',
'https://upload.wikimedia.org/wikipedia/commons/2/27/Flag_of_Baja_California.svg',
'https://upload.wikimedia.org/wikipedia/commons/0/01/Flag_of_California.svg',
'https://upload.wikimedia.org/wikipedia/commons/4/46/Flag_of_Colorado.svg',
'https://upload.wikimedia.org/wikipedia/commons/f/f5/Flag_of_Cross_of_Burgundy.svg',
'https://upload.wikimedia.org/wikipedia/commons/f/f1/Flag_of_Nevada.svg',
'https://upload.wikimedia.org/wikipedia/commons/c/c3/Flag_of_New_Mexico.svg',
'https://upload.wikimedia.org/wikipedia/commons/6/69/Flag_of_Sonora.svg',
'https://upload.wikimedia.org/wikipedia/commons/7/70/Flag_of_the_State_of_Utah.svg',
'https://upload.wikimedia.org/wikipedia/commons/5/5c/Flag_of_Baden-W%C3%BCrttemberg.svg',
'https://upload.wikimedia.org/wikipedia/commons/1/1f/Flag_of_Kosovo.svg',
'https://upload.wikimedia.org/wikipedia/commons/5/53/Flag_of_Syria.svg',
'https://upload.wikimedia.org/wikipedia/en/0/03/Flag_of_Italy.svg',
'https://upload.wikimedia.org/wikipedia/en/1/12/Flag_of_Poland.svg',
'https://upload.wikimedia.org/wikipedia/commons/e/ed/Flag_of_Guinea.svg',
'https://upload.wikimedia.org/wikipedia/commons/9/99/Flag_of_Guyana.svg',
'https://upload.wikimedia.org/wikipedia/commons/9/9f/Flag_of_Indonesia.svg',
'https://upload.wikimedia.org/wikipedia/commons/0/0a/Flag_of_Jamaica.svg',
'https://upload.wikimedia.org/wikipedia/commons/2/21/Flag_of_Vietnam.svg',
'https://upload.wikimedia.org/wikipedia/commons/archive/b/b5/20210810162633%21Radioactive.svg',
'https://upload.wikimedia.org/wikipedia/commons/archive/b/b5/20210730155829%21Radioactive.svg',
'https://upload.wikimedia.org/wikipedia/commons/archive/b/b5/20201008113333%21Radioactive.svg',
'https://upload.wikimedia.org/wikipedia/commons/archive/b/b5/20170417055323%21Radioactive.svg',
'https://upload.wikimedia.org/wikipedia/commons/archive/b/b5/20150903090719%21Radioactive.svg',
'https://upload.wikimedia.org/wikipedia/commons/archive/b/b5/20121027191728%21Radioactive.svg',
'https://upload.wikimedia.org/wikipedia/commons/archive/b/b5/20070331090142%21Radioactive.svg',
'https://upload.wikimedia.org/wikipedia/commons/archive/b/b5/20060120180222%21Radioactive.svg',
'https://upload.wikimedia.org/wikipedia/commons/archive/b/b5/20060119161202%21Radioactive.svg',
'https://upload.wikimedia.org/wikipedia/commons/archive/b/b5/20060119160714%21Radioactive.svg'
'https://upload.wikimedia.org/wikipedia/commons/b/b1/Ambox_outdated_content.svg',
'https://upload.wikimedia.org/wikipedia/en/9/9c/Symbol_file_class.svg',
'https://upload.wikimedia.org/wikipedia/commons/7/75/DEU_Landau_in_der_Pfalz_COA.svg',
'https://upload.wikimedia.org/wikipedia/commons/f/f4/Image-Blason_Colmar_68.svg',
'https://upload.wikimedia.org/wikipedia/commons/d/dd/Greater_coat_of_arms_of_Strasbourg.svg',
'https://upload.wikimedia.org/wikipedia/commons/2/2e/FRA_Obernai_67_COA.svg',
'https://upload.wikimedia.org/wikipedia/commons/c/c7/Flag_of_Alsace.svg',
'https://upload.wikimedia.org/wikipedia/commons/1/1a/Flag_of_Alsace_%28historical%29.svg',
'https://upload.wikimedia.org/wikipedia/commons/f/f1/Flag_of_Alsace_%28old%29.svg',
'https://upload.wikimedia.org/wikipedia/commons/4/42/Alsace_flag_waving_icon.svg',
'https://upload.wikimedia.org/wikipedia/commons/d/df/Alsaceregionsnaturelles.jpg',
'https://upload.wikimedia.org/wikipedia/commons/6/63/Arms_of_Alsace_%281663%29.svg',
'https://upload.wikimedia.org/wikipedia/commons/c/c4/Arrondissements_Bas-Rhin.svg',
'https://upload.wikimedia.org/wikipedia/commons/d/d2/Arrondissements_Haut-Rhin.svg',
'https://upload.wikimedia.org/wikipedia/commons/3/3f/Banner_of_the_Holy_Roman_Emperor_%28after_1400%29.svg',
'https://upload.wikimedia.org/wikipedia/commons/e/ee/BlasonAlsace.svg',
'https://upload.wikimedia.org/wikipedia/commons/f/f8/Blason_Mulhouse.svg',
'https://upload.wikimedia.org/wikipedia/commons/2/21/Blason_Rosheim_67.svg',
'https://upload.wikimedia.org/wikipedia/commons/7/79/Blason_Wissembourg_67.svg',
'https://upload.wikimedia.org/wikipedia/commons/b/be/Blason_de_la_ville_de_Kaysersberg_%2868%29.svg',
'https://upload.wikimedia.org/wikipedia/commons/e/ec/Blason_de_la_ville_de_Munster_%2868%29.svg',
'https://upload.wikimedia.org/wikipedia/commons/5/5e/Blason_de_la_ville_de_Turckheim_%2868%29.svg',
'https://upload.wikimedia.org/wikipedia/commons/f/fa/Blason_haguenau_67.svg',
'https://upload.wikimedia.org/wikipedia/commons/2/27/Blason_ville_fr_Seltz.svg',
'https://upload.wikimedia.org/wikipedia/commons/5/5e/Blason_ville_fr_S%C3%A9lestat_%28Alsace%29.svg',
'https://upload.wikimedia.org/wikipedia/commons/c/cc/Flag_of_Antofagasta_Region%2C_Chile.svg',
'https://upload.wikimedia.org/wikipedia/commons/8/8b/Logotipo_de_la_Gobernaci%C3%B3n_de_El_Loa.svg',
'https://upload.wikimedia.org/wikipedia/commons/1/1a/Mapa_loc_Antofagasta.svg',
'https://upload.wikimedia.org/wikipedia/commons/6/6a/Provincia_de_El_Loa.svg',
'https://upload.wikimedia.org/wikipedia/commons/0/0c/Red_pog.svg',
'https://upload.wikimedia.org/wikipedia/commons/5/59/Seal_of_the_President_of_the_Philippines.svg',
'https://upload.wikimedia.org/wikipedia/commons/3/35/Martial_arts_film_clapperboard.svg',
'https://upload.wikimedia.org/wikipedia/commons/5/51/Star_full.svg',
'https://upload.wikimedia.org/wikipedia/commons/8/81/Star_half.svg',
'https://upload.wikimedia.org/wikipedia/commons/5/5c/Flag_of_Alabama.svg',
'https://upload.wikimedia.org/wikipedia/commons/2/2e/Map_of_Alabama_highlighting_Jefferson_County.svg',
'https://upload.wikimedia.org/wikipedia/commons/0/0c/Red_pog.svg',
'https://upload.wikimedia.org/wikipedia/commons/9/93/USA_Alabama_location_map.svg',
'https://upload.wikimedia.org/wikipedia/commons/2/20/Usa_edcp_location_map.svg',
'https://upload.wikimedia.org/wikipedia/commons/9/9d/Cu-pourbaix-diagram.svg',
'https://upload.wikimedia.org/wikipedia/commons/c/c9/Cubic-face-centered.svg',
'https://upload.wikimedia.org/wikipedia/en/a/a4/Flag_of_the_United_States.svg',
'https://upload.wikimedia.org/wikipedia/commons/6/6f/Colorado_geographic_map-en.svg',
'https://upload.wikimedia.org/wikipedia/commons/6/60/Colorado_in_United_States.svg',
'https://upload.wikimedia.org/wikipedia/en/archive/9/99/20210726203439%21Question_book-new.svg',
'https://upload.wikimedia.org/wikipedia/en/archive/9/99/20201004174738%21Question_book-new.svg',
'https://upload.wikimedia.org/wikipedia/en/archive/9/99/20171208221057%21Question_book-new.svg',
'https://upload.wikimedia.org/wikipedia/en/archive/9/99/20171207131032%21Question_book-new.svg',
'https://upload.wikimedia.org/wikipedia/en/archive/9/99/20160612140736%21Question_book-new.svg',
'https://upload.wikimedia.org/wikipedia/commons/2/29/Books-aj.svg_aj_ashton_01b.svg',
'https://upload.wikimedia.org/wikipedia/en/e/e2/Symbol_portal_class.svg',
'https://upload.wikimedia.org/wikipedia/commons/4/49/Star_empty.svg',
'https://upload.wikimedia.org/wikipedia/commons/5/51/Star_full.svg',
'https://upload.wikimedia.org/wikipedia/commons/a/a8/Maldonado_in_Uruguay.svg',
'https://upload.wikimedia.org/wikipedia/commons/0/0c/Red_pog.svg',
'https://upload.wikimedia.org/wikipedia/commons/c/c5/Uruguay_location_map.svg',
'https://upload.wikimedia.org/wikipedia/commons/4/49/Star_empty.svg',
'https://upload.wikimedia.org/wikipedia/commons/5/51/Star_full.svg',
'https://upload.wikimedia.org/wikipedia/commons/8/81/Star_half.svg',
'https://upload.wikimedia.org/wikipedia/commons/7/70/Bandera_de_la_Ciudad_de_Maldonado.svg',
'https://upload.wikimedia.org/wikipedia/commons/0/0c/Red_pog.svg',
'https://upload.wikimedia.org/wikipedia/en/a/a4/Flag_of_the_United_States.svg',
'https://upload.wikimedia.org/wikipedia/commons/e/ea/Chihuahua_en_Chihuahua.svg',
'https://upload.wikimedia.org/wikipedia/commons/a/ae/Chihuahua_en_M%C3%A9xico.svg',
'https://upload.wikimedia.org/wikipedia/commons/a/ab/Flag_of_Chihuahua%2C_Chihuahua.svg',
'https://upload.wikimedia.org/wikipedia/commons/1/1b/Flag_of_Chihuahua.svg',
'https://upload.wikimedia.org/wikipedia/commons/5/5c/PAN_logo_%28Mexico%29.svg',
'https://upload.wikimedia.org/wikipedia/commons/5/5c/PAN_logo_%28Mexico%29.svg',
'https://upload.wikimedia.org/wikipedia/commons/7/79/Map_of_Indiana_highlighting_Dubois_County.svg',
'https://upload.wikimedia.org/wikipedia/commons/0/0c/Red_pog.svg',
'https://upload.wikimedia.org/wikipedia/commons/3/3f/USA_Indiana_location_map.svg',
'https://upload.wikimedia.org/wikipedia/commons/2/20/Usa_edcp_location_map.svg',
'https://upload.wikimedia.org/wikipedia/commons/d/d6/TOSC_Order_of_Santo_Stefano_BAR.svg',
'https://upload.wikimedia.org/wikipedia/commons/e/eb/Teutonic_Order_BAR.svg',
'https://upload.wikimedia.org/wikipedia/commons/0/04/VA_Ordine_Piano_BAR.svg',
'https://upload.wikimedia.org/wikipedia/commons/3/30/Cavaliere_BAR.svg',
'https://upload.wikimedia.org/wikipedia/commons/d/d6/Cavaliere_OCI_BAR.svg',
'https://upload.wikimedia.org/wikipedia/commons/9/9c/Cavaliere_OML_BAR.svg',
'https://upload.wikimedia.org/wikipedia/commons/d/dc/Cavaliere_OMRI_BAR.svg',
'https://upload.wikimedia.org/wikipedia/commons/c/c0/Cavaliere_SSML_BAR.svg',
'https://upload.wikimedia.org/wikipedia/commons/0/00/Flag_of_the_Vatican_City.svg',
'https://upload.wikimedia.org/wikipedia/commons/7/79/P_christianity.svg',
'https://upload.wikimedia.org/wikipedia/commons/b/bb/SPQRomani.svg',
'https://upload.wikimedia.org/wikipedia/en/6/69/P_vip.svg',
'https://upload.wikimedia.org/wikipedia/commons/4/48/Gloriole_blur.svg',
'https://upload.wikimedia.org/wikipedia/commons/7/79/P_christianity.svg',
'https://upload.wikimedia.org/wikipedia/commons/4/48/P_history.svg',
'https://upload.wikimedia.org/wikipedia/en/e/e2/Symbol_portal_class.svg',
'https://upload.wikimedia.org/wikipedia/commons/1/1f/Escudo_de_Calama.svg',
'https://upload.wikimedia.org/wikipedia/commons/f/f6/Chile_location_map.svg',
'https://upload.wikimedia.org/wikipedia/commons/c/c5/Comuna_de_Calama.svg',
'https://upload.wikimedia.org/wikipedia/commons/9/98/Tree_template.svg',
'https://upload.wikimedia.org/wikipedia/commons/0/03/Rhombohedral.svg',
'https://upload.wikimedia.org/wikipedia/commons/7/79/Bismuth_symbol.svg'
]

In [18]:
# Criaremos um laço de comparação entre a lista de url e a lista de termos para excluir os termos indesejados

for i in range (0, len(url_img)):

 for attempt in range(0, 3):
  
  try:

    for a in range (0, len(url_img[i])):

      for term in range (0, len(terms)):

        if url_img[i][a] == terms[term]:

         del(url_img[i][a])

  except:

      pass

A segunda maneira é um pouco mais inteligente. Passaremos todos os elementos da lista url_img para uma outra lista urls e eliminaremos elementos repetidos

In [19]:
# Podemos separar todos esses elementos e colocar dentro de uma lista

urls = []


for objects in range (0, len(url_img)):
  for element in range (0, len(url_img[objects])):
    urls.append(url_img[objects][element])

In [20]:
# Iremos ordenar e remover os termos repetidos da lista

set_url = sorted(set(urls))
list_url = list(set_url)

#len(set_url)

## **Criando dataframe**

In [21]:
# Criando os dataframes

df_urls01 = pd.DataFrame(list_url, columns=['url'])
df_urls02 = pd.DataFrame(url_img)

In [24]:
# Baixando os dataframes

df_urls01.to_csv("url_images_01.csv", sep =',', index=False)
df_urls02.to_csv("url_images_02.csv", sep =',', index=False)